In [1]:
# reload magic 
%load_ext autoreload

In [2]:
import stim

In [3]:
def index_vertex(x, y, L):
    return x + y * L


def num_pauli_to_str(idx, n, paulitype):
    """
    Convert a number to a string of Pauli operators.
    """
    pauli = ["_"] * n
    for i in idx:
        pauli[i] = paulitype
    return "".join(pauli)

def generate_rotated_surface_code_stabilizers(L):
    """
    construct the set of stabilizers for the rotated toric code having periodic boundary conditions.
    """
    x_stabilizers = []
    z_stabilizers = []
    n = L**2  # total number of qubits in the standard toric code layout

    # For each vertex (x, y), define a star operator: product of X on the 4 edges touching that vertex.
    # Edges: (x,y) horiz, (x-1,y) horiz, (x,y) vert, (x,y-1) vert, all mod L for periodic BCs.
    for x in range(L):
        for y in range(L):
            v1 = index_vertex(x, y, L)
            if x == L - 1 or y == L - 1:
                continue
            v2 = index_vertex(x, (y + 1), L)
            v3 = index_vertex((x + 1), y, L)
            v4 = index_vertex((x + 1), (y + 1), L)
            vertices = [v1, v2, v3, v4]
            if ( x + y ) % 2 == 0:
                ptype = "X"
                x_stabilizers.append(vertices)
            else:
                ptype = "Z"     
                z_stabilizers.append(vertices)

    # top and bottom
    for x in range(L):
        if x < L - 1:
            ptype = "Z"
            if x % 2 == 0 :
                v1 = index_vertex(x, 0, L)
                v2 = index_vertex(x + 1, 0, L)
                z_stabilizers.append([v1, v2])
            elif x % 2 == 1:
                v1 = index_vertex(x, L - 1, L)
                v2 = index_vertex(x + 1, L - 1, L)
                z_stabilizers.append([v1, v2])

    for y in range(L):
        if y < L - 1:
            ptype = "X"
            if y % 2 == 1:
                v1 = index_vertex(0, y, L)
                v2 = index_vertex(0, y + 1, L)
                x_stabilizers.append([v1, v2])
            elif y % 2 == 0:
                v1 = index_vertex(L - 1, y, L)
                v2 = index_vertex(L - 1, y + 1, L)
                x_stabilizers.append([v1, v2])
    print(x_stabilizers)
    print(z_stabilizers)
    stabilizers = x_stabilizers + z_stabilizers
    stabilizers = [num_pauli_to_str(stabilizer, n, ptype) for stabilizer in stabilizers]
    stabilizers = [stim.PauliString(stabilizer) for stabilizer in stabilizers]
    return stabilizers


In [4]:
L = 3
stabilizers = generate_rotated_surface_code_stabilizers(L)
display(stabilizers)

[[0, 3, 1, 4], [4, 7, 5, 8], [2, 5], [3, 6]]
[[3, 6, 4, 7], [1, 4, 2, 5], [0, 1], [7, 8]]


[stim.PauliString("+XX_XX____"),
 stim.PauliString("+____XX_XX"),
 stim.PauliString("+__X__X___"),
 stim.PauliString("+___X__X__"),
 stim.PauliString("+___XX_XX_"),
 stim.PauliString("+_XX_XX___"),
 stim.PauliString("+XX_______"),
 stim.PauliString("+_______XX")]

In [7]:
t = stim.Tableau.from_stabilizers(stabilizers, allow_redundant=True, allow_underconstrained=True)
# we leave out the final stabilizer that canonically represents the degree of 
# freedom for a state we didn't specify
generators_S = [t.z_output(k) for k in range(len(t) - 1)]
generators_T = [t.x_output(k) for k in range(len(t) - 1)]
generators_L = [stim.PauliString(p*L) for p in ["I", "X"]]

In [ ]:
from mldec.utils import pauli_decoding_stim
p_I = 0.99
n = L ** 2
k = 1
temp = []
pvec = [p_I, (1-p_I)/3, (1-p_I)/3, (1-p_I)/3]
error_channel = lambda x: pauli_decoding_stim.pauli_vector_channel(pvec, x, n)
p_SA = pauli_decoding_stim.compute_coset_distribution_pure_errors(generators_S, generators_T, generators_L, error_channel)

mle = pauli_decoding_stim.mle_success_prob(p_SA)
print("Maximum likelihood, L={}".format(L))
print(mle)


8 9 1


Maximum likelihood, L=3
0.9397690759087137


In [11]:
from itertools import chain, combinations
from functools import reduce
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
        


In [15]:
enumerate(powerset([1,2,3]))

#### what we need is:
1. answers for MLE on every syndrome -> separates good and bad
2. for all "good" syndromes, we need answers from pymatching.

#### build the parity check matrices

so MWPM decoding for X-type and Z-type errors is the same process (PCM might be different), so our procedure is:
1. get the syndrome logical pair $(\sigma, \ell)$
2. Check that $f^*(\sigma) = \ell$, so we know we have a good example
3. Let $\sigma = [\sigma_X | \sigma_Z]$
4. MWPM decode $\sigma$ into $\ell_X \ell_Z = \hat{\ell}$, our guess for the logical operator
5. if $\hat{\ell} \notin \ell \mathcal{S}$, then we have an important example

Issues:
 - organizing everything might be tough; do we start with $\ell$
 - idea: I build the prism from the ground up; I could run MWPM at each layer before generating $\ell \mathcal{S}$, then check equality as I fill out that coset
 - probably need to downgrade to numpy 1.26

In [19]:
import numpy as np
from scipy.sparse import hstack, kron, eye, csc_matrix, block_diag


def repetition_code(n):
    """
    Parity check matrix of a repetition code with length n.
    """
    row_ind, col_ind = zip(*((i, j) for i in range(n) for j in (i, (i+1)%n)))
    data = np.ones(2*n, dtype=np.uint8)
    return csc_matrix((data, (row_ind, col_ind)))


def toric_code_x_stabilisers(L):
    """
    Sparse check matrix for the X stabilisers of a toric code with
    lattice size L, constructed as the hypergraph product of
    two repetition codes.
    """
    Hr = repetition_code(L)
    H = hstack(
            [kron(Hr, eye(Hr.shape[1])), kron(eye(Hr.shape[0]), Hr.T)],
            dtype=np.uint8
        )
    H.data = H.data % 2
    H.eliminate_zeros()
    return csc_matrix(H)



In [ ]:
toric_code